# Carregando Pacotes:

In [1]:
#pip install pdfplumber

In [2]:
from transformers import pipeline
import torch
import pdfplumber
from transformers import AutoTokenizer

import modelo_eqa as eqa



In [3]:
# Verifica e configura o dispositivo (GPU/CPU)
device = 0 if torch.cuda.is_available() else -1  # 0 = GPU, -1 = CPU
print(f"🔧 Rodando em: {'GPU' if device == 0 else 'CPU'}")

🔧 Rodando em: GPU


# Funções:

In [4]:
def carregar_documento(caminho_do_pdf):

    with pdfplumber.open(caminho_do_pdf) as pdf:
        texto = " ".join(
            page.extract_text() for page in pdf.pages
            if page.extract_text()
        )
        texto = " ".join(texto.split())
    return texto

In [5]:
def dividir_em_chunks_tokenizados(texto):

    tokens = tokenizer.tokenize(texto)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        chunks.append(tokenizer.convert_tokens_to_string(chunk))
    return chunks

In [6]:
def sliding_window_tokenizados(texto):
    tokens = tokenizer.tokenize(texto)
    janelas = []

    for i in range(0, len(tokens), stride):
        janela = tokens[i:i + window_size]
        texto_janela = tokenizer.convert_tokens_to_string(janela)
        janelas.append(texto_janela)

        if i + window_size >= len(tokens):
            break

    return janelas

In [7]:
def processar_pergunta(pergunta, documento):

    chunks = divisao_do_texto

    # 1. Tokeniza a pergunta para verificar tamanho
    tokens_pergunta = tokenizer.tokenize(pergunta)
    if len(tokens_pergunta) > tamanho_pergunta:  # Limite arbitrário (ajuste conforme necessário)
        print("Pergunta muito longa! Simplifique para melhor precisão.")

    # 2. Executa Q&A em cada chunk
    respostas = []
    for chunk in chunks:
        try:
            resposta = qa_pipeline(question=pergunta, context=chunk)
            respostas.append(resposta)

            resposta_completa = {
                'answer': resposta.get('answer', ''),
                'score': resposta.get('score', 0),
                'context': resposta.get('context', chunk[:500]),  # Fallback: 500 primeiros chars
                'chunk_completo': chunk
            }
            respostas.append(resposta_completa)
        except Exception as e:
            print(f"Erro no chunk: {str(e)}")
            continue

    # 3. Filtra respostas com score baixo e seleciona a melhor
    respostas_validas = [r for r in respostas if r['score'] >= 0.2]
    if not respostas_validas:
        print("Não há resposta sobre isto nesse documento.")
        return None
    return max(respostas_validas, key=lambda x: x['score'])

In [8]:
import re

def extrair_trecho_com_palavras(texto, resposta, palavras_ao_redor=30):
    # Encontra a posição da resposta no texto
    start_idx = texto.find(resposta)
    if start_idx == -1:
        return texto.split()[:palavras_ao_redor]  # Fallback: primeiras 30 palavras
    
    # Extrai palavras antes e depois da resposta
    palavras = texto.split()
    palavras_resposta = resposta.split()
    
    # Encontra o índice aproximado da resposta no texto dividido por palavras
    for i in range(len(palavras) - len(palavras_resposta) + 1):
        if palavras[i:i+len(palavras_resposta)] == palavras_resposta:
            start_word_idx = i
            break
    else:
        return " ".join(palavras[:palavras_ao_redor])  # Fallback
    
    # Calcula os índices do trecho desejado
    inicio = max(0, start_word_idx - palavras_ao_redor)
    fim = min(len(palavras), start_word_idx + len(palavras_resposta) + palavras_ao_redor)
    
    return " ".join(palavras[inicio:fim])


# Tokenizador

In [48]:
#Portugues
# modelo_tokenizador = "mrm8488/distilbert-multi-finedtuned-squad-pt"
modelo_tokenizador = "pierreguillou/bert-large-cased-squad-v1.1-portuguese"
# modelo_tokenizador = "pierreguillou/bert-base-cased-squad-v1.1-portuguese"

#Multilingual
# modelo_tokenizador = "Khanh/bert-base-multilingual-cased-finetuned-squad"

#Ingles
# modelo_tokenizador = "deepset/roberta-base-squad2"

# modelo_tokenizador = "deepset/roberta-large-squad2"

# modelo_tokenizador = "distilbert/distilbert-base-uncased-distilled-squad"

# modelo_tokenizador = "sjrhuschlee/flan-t5-base-squad2"

# modelo_tokenizador = "deepset/electra-base-squad2"

# modelo_tokenizador = "deepset/bert-large-uncased-whole-word-masking-squad2"

# modelo_tokenizador = "deepset/bert-base-uncased-squad2"

# Variáveis Globais

In [49]:
#Tamanho da Janela de Contexto
window_size=400

#Overlap da Janela de Contexto
stride=100

#Tamanho da Chunk
chunk_size = 400

#Overlap da Chunk
overlap = 100

#Tamanho Máximo da Pergunta
tamanho_pergunta = 60

In [50]:
# Carregar modelo BERTimbau pré-treinado para Q&A em Portugues
tokenizer = AutoTokenizer.from_pretrained(modelo_tokenizador)

qa_pipeline = pipeline(
    "question-answering",
    model= modelo_tokenizador,  # Modelo em Portugues
    tokenizer=tokenizer,
    device=device
)

Device set to use cuda:0


# Processamento

In [51]:
#Carrega o documento em PDF

documento = carregar_documento("Cabelo Cacheado Maior.pdf")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [52]:
#Divisao em Janela Deslizante

# divisao_do_texto = sliding_window_tokenizados(documento)

#Divisao em Chunks

divisao_do_texto = dividir_em_chunks_tokenizados(documento)

# Divisão em Chunks

In [53]:
def dividir_em_chunks_com_palavras(texto):
    tokens = tokenizer.tokenize(texto)  # Tokeniza o texto
    chunks_info = []  # Armazenará os chunks e suas palavras
    
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk_tokens = tokens[i:i + chunk_size]
        chunk_texto = tokenizer.convert_tokens_to_string(chunk_tokens)  # Converte para string
        
        # Extrai as 3 primeiras e últimas PALAVRAS (não tokens)
        palavras = chunk_texto.split()  # Divide o chunk em palavras (por espaços)
        
        primeiras = ' '.join(palavras[:3]) if len(palavras) >= 3 else ' '.join(palavras)
        ultimas = ' '.join(palavras[-3:]) if len(palavras) >= 3 else ' '.join(palavras)
        
        chunks_info.append({
            'chunk_completo': chunk_texto,
            'inicio': primeiras,
            'fim': ultimas
        })
    
    return chunks_info

In [54]:
# Divide o texto e pega palavras iniciais/finais
chunks = dividir_em_chunks_com_palavras(documento)

# Exemplo de saída para o primeiro chunk
for i in range(len(chunks)):
    print(f"Chunk {i}")
    print("3 primeiras palavras:", chunks[i]['inicio'])
    print("3 últimas palavras:", chunks[i]['fim'])

Chunk 0
3 primeiras palavras: A Definição e
3 últimas palavras: ressaltar que essa
Chunk 1
3 primeiras palavras: ##ículo, mais apertada
3 últimas palavras: em comunidades negras,
Chunk 2
3 primeiras palavras: O Tipo 4,
3 últimas palavras: atrito que causa
Chunk 3
3 primeiras palavras: exemplo, é amplamente
3 últimas palavras: biológica, cultural ou
Chunk 4
3 primeiras palavras: o " devacut
3 últimas palavras: que eles representam.


# Divisão em Janelas Deslizantes

In [55]:
def sliding_window_com_palavras(texto):
    tokens = tokenizer.tokenize(texto)
    janelas = []

    for i in range(0, len(tokens), stride):
        janela_tokens = tokens[i:i + window_size]
        texto_janela = tokenizer.convert_tokens_to_string(janela_tokens)  # Converte para string
        
        # Extrai as 3 primeiras e últimas PALAVRAS (não tokens)
        palavras = texto_janela.split()  # Divide o texto da janela em palavras
        
        primeiras = ' '.join(palavras[:3]) if len(palavras) >= 3 else ' '.join(palavras)
        ultimas = ' '.join(palavras[-3:]) if len(palavras) >= 3 else ' '.join(palavras)
        
        janelas.append({
            'janela_completa': texto_janela,
            'inicio_3_palavras': primeiras,
            'fim_3_palavras': ultimas
        })

        if i + window_size >= len(tokens):
            break

    return janelas

In [57]:
# Divide o texto e pega palavras iniciais/finais
janelas = sliding_window_com_palavras(documento)

# Exemplo de saída para o primeiro chunk
for i in range(len(janelas)):
    print(f"Chunk {i}")
    print("3 primeiras palavras:", janelas[i]['inicio_3_palavras'])
    print("3 últimas palavras:", janelas[i]['fim_3_palavras'])

Chunk 0
3 primeiras palavras: A Definição e
3 últimas palavras: ressaltar que essa
Chunk 1
3 primeiras palavras: variados quanto as
3 últimas palavras: ), que já
Chunk 2
3 primeiras palavras: crescem a partir
3 últimas palavras: técnicas de finalização.
Chunk 3
3 primeiras palavras: ##ículo, mais apertada
3 últimas palavras: em comunidades negras,
Chunk 4
3 primeiras palavras: categorização não é
3 últimas palavras: os óleos naturais
Chunk 5
3 primeiras palavras: começam a formar
3 últimas palavras: Oil ), por
Chunk 6
3 primeiras palavras: O Tipo 4,
3 últimas palavras: atrito que causa
Chunk 7
3 primeiras palavras: onde o cabelo
3 últimas palavras: extremos, permanentemente. Cultural
Chunk 8
3 primeiras palavras: produzidos pelo couro
3 últimas palavras: cachos ( como
Chunk 9
3 primeiras palavras: exemplo, é amplamente
3 últimas palavras: biológica, cultural ou
Chunk 10
3 primeiras palavras: o frizz. A
3 últimas palavras: que eles representam.
